In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function    # (at top of module)
import os, sys
import argparse
import pprint
import tempfile
import subprocess
import timeit
import shutil
import datetime
from subprocess import check_output, call

# Python 2 and 3:

pp = pprint.PrettyPrinter(indent=4)

def critical(message):
    print(message)
    exit(1)
def warning(message):
    print(message)
    
class VerboseHelper():
    verbose_flag = False
    @staticmethod
    def info(message):
        if VerboseHelper.verbose_flag:
            print(message)
def verbose(message):
    VerboseHelper.info(message)
    
def execute(*args, **options):
    ret_code = call(*args, **options)
    if ret_code:
        raise RuntimeWarning('operation failed {}'.format(str(*args) + ' ' + str(dict(**options))))
    return
    process = subprocess.Popen(*args, **options, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Poll process for new output until finished
    while True:
        nextline = process.stdout.readline()
        if nextline == '' and process.poll() is not None:
            break
        sys.stdout.write(nextline)
        sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return output
    else:
        raise ProcessException(command, exitCode, output)
def find_existent(*args):
    for d in list(args):
        if os.path.exists(d):
            return d
    # else not exists
    raise FileExistsError(str(list(args)))
    return ''
    
# --- VARIABLES ---
branch = 'origin/master'
first_commit_date = "2018-08-10 13:37"

project_dir_nb = 'C:/Projects/cppcheck'
project_dir_pc = 'D:/Study/TestCoveredProjects/cppcheck_copy'
project_dir = find_existent(project_dir_nb, project_dir_pc)

cmake_cxx_compiler_nb = 'C:/Qt/Tools/mingw530_32/bin/g++.exe'
cmake_cxx_compiler_pc = 'D:/Soft/Qt/Tools/mingw530_32/bin/g++.exe'
cmake_cxx_compiler = find_existent(cmake_cxx_compiler_nb, cmake_cxx_compiler_pc)

lazyut_nb = 'C:/Projects/Study/build-LazyRT-5_9-Default/bin/lazyut.exe'
lazyut_pc = 'D:/Study/build-LazyUT-5_7-Debug/bin/lazyut.exe'
lazyut = find_existent(lazyut_nb, lazyut_pc)

# --- CODE ---
def prepareCppcheck(pro_dir):
    cmakelists_test = os.path.join(pro_dir, 'test', 'CMakeLists.txt')
    # We are going to modify file, so will use temporary file
    tmpf = tempfile.NamedTemporaryFile(mode='r+')
    # Open for read
    with open(cmakelists_test, 'r') as cmakelists_txt:
        # Firstly check if it is not prepared yet
        # And return if it is
        for line in cmakelists_txt:
            if 'shlwapi' in line:
                tmpf.close()
                return 
        cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
        # Create temporary file read/write
        # Copy input file to temporary file, modifying as we go
        for line in cmakelists_txt:
            tmpf.write(line)
            # Link missed libraries (MINGW)
            if 'add_executable(testrunner' in line:
                tmpf.write('    if (MINGW)\n')
                tmpf.write('        target_link_libraries(testrunner shlwapi)\n')
                tmpf.write('    endif()\n')
#                 tmpf.write('    message(srcs : ${srcs})\n')
    tmpf.seek(0) # Rewind temporary file to beginning
    # Reopen for write
    with open(cmakelists_test, 'w') as cmakelists_txt:
        for line in tmpf:
            cmakelists_txt.write(line)
    tmpf.close()

def removeFilesLazyUT(pro_dir):
    lazyut_files_dir = os.path.join(pro_dir, 'test', 'lazyut')
    shutil.rmtree(lazyut_files_dir, ignore_errors=True) 

def moveFilesLazyUT(pro_dir):
    # initialize paths to directories
    lazyut_dir = os.path.join(pro_dir, 'test', 'lazyut')
    lazyut_files_dir = os.path.join(lazyut_dir, 'files')
    lazyut_last_version_dir = os.path.join(lazyut_dir, 'last_version', 'files')
    # get list of files
    files = os.listdir(lazyut_files_dir)
    # create last_version/files/ directory
    os.makedirs(lazyut_last_version_dir, exist_ok=True)
    # move files to last_version/files/ directory and overwrite if exists
    for f in files:
        shutil.move(os.path.join(lazyut_files_dir, f), os.path.join(lazyut_last_version_dir, f))
    
def run_lazyut(pro_dir, info = None):
    srcdir = os.path.join(pro_dir, 'lib')
    testdir = os.path.join(pro_dir, 'test')
    outdir = os.path.join(testdir, 'lazyut', 'files')
    indir = os.path.join(testdir, 'lazyut', 'last_version', 'files')
    analysis_sdt = timeit.default_timer()
    execute(lazyut + ' -s {} -t {} -o {} -i {} --ignore="cfg/,cfg\,synthetic/,synthetic\"'.format(
            srcdir, testdir, outdir, indir), shell = True)
    analysis_edt = timeit.default_timer()
    if info:
        info.setDurationLazyutAnalysis(analysis_edt - analysis_sdt)
    
    
def prepareLazyUT(pro_dir, method, info = None):
    cmakelists_test = os.path.join(pro_dir, 'test', 'CMakeLists.txt')
    # We are going to modify file, so will use temporary file
    tmpf = tempfile.NamedTemporaryFile(mode='r+')
    # Open for read
    with open(cmakelists_test, 'r') as cmakelists_txt:
        # Firstly check if it is not prepared yet
        # And return if it is
        for line in cmakelists_txt:
            if 'lazyut' in line:
                return
        cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
        # Create temporary file read/write
        # Copy input file to temporary file, modifying as we go
        for line in cmakelists_txt:
            method(line, tmpf)
    tmpf.seek(0) # Rewind temporary file to beginning
    # Reopen for write
    with open(cmakelists_test, 'w') as cmakelists_txt:
        for line in tmpf:
            cmakelists_txt.write(line)
    run_lazyut(pro_dir, info = info)
            
def CMAKE_GLOB_SRCS(line, tmpf):
    # Add only affected test files
    if 'file(GLOB srcs' in line:
        tmpf.write('    file(STRINGS lazyut/files/tests_affected.txt srcs)\n')
        return
    if 'file(GLOB hdrs' in line:
        tmpf.write('    set(hdrs "") # emptry list\n')
        return
    # Just copy line
    tmpf.write(line)
                
                
class TestMode:
    unknown = 0
    retest_all = 1
    test_affected = 2
    @staticmethod
    def name(mode):
        if mode == TestMode.unknown:
            return ''
        if mode == TestMode.retest_all:
            return 're-test all'
        if mode == TestMode.test_affected:
            return 'lazy testings'
    @staticmethod
    def common_directory():
        return 'default'
    @staticmethod
    def directory(mode):
        if mode == TestMode.unknown:
            return ''
        if mode == TestMode.retest_all:
            return 'default'
        if mode == TestMode.test_affected:
            return 'lazyut'
    @staticmethod
    def isLazyUT(mode):
        return mode == TestMode.test_affected;
            
def clear_out_file():
    with open('output.txt', 'w'): 
        pass
    
def append_out_file(line):
    with open('output.txt', 'a+') as f: 
        f.write(line)
        f.write('\n')
def is_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) == 0

def build_project(pro_dir='', test_mode=TestMode.unknown, info = None):
    build_dir = os.path.join(pro_dir, 'build', TestMode.common_directory())
    # clear & delete if exists build directory
#     shutil.rmtree(build_dir, ignore_errors=True) 
    # create build directory
    os.makedirs(build_dir, exist_ok=True)
    # setup project
    prepareCppcheck(pro_dir)
    if TestMode.isLazyUT(test_mode):
        prepareLazyUT(pro_dir, CMAKE_GLOB_SRCS, info = info)
        affected_tests_txt = os.path.join(pro_dir, 'lazyut', 'files', 'tests_affected.txt')
        if is_zero_file(affected_tests_txt):
            return
    # run cmake
    sdt = timeit.default_timer()
    try:
        execute('cmake -G "Unix Makefiles" -DCMAKE_CXX_COMPILER={0} -DBUILD_TESTS=ON {1}'
                .format(cmake_cxx_compiler, project_dir), cwd = build_dir, shell = True)
    except:
        message = 'cmake failed!'
        if info:
            message += ' ' + info.commit
        print(message)
        return
    cmake_edt = timeit.default_timer()
    try:
        execute('make', timeout=None, cwd = build_dir, shell = True)
    except:
        
        message = 'Make failed!'
        if info:
            message += ' ' + info.commit
        print(message)
        return
    make_edt = timeit.default_timer()
    testrunner = 'testrunner.exe'
    testrunner_dir = os.path.join(build_dir, 'bin')
    try:
        execute(testrunner, cwd = testrunner_dir, shell=True)
    except:
        message = 'Tests failed!'
        if info:
            info.addFailed(test_mode)
            message += ' ' + info.commit
        print(message)
        return
    test_edt = timeit.default_timer()
    test_duration = test_edt - make_edt
    output = '{}: cmake [{:.5}] make [{:.5}] testing [{:.5}]'.format(
        TestMode.name(test_mode), cmake_edt - sdt, make_edt - cmake_edt, test_duration)
    print(output)
    append_out_file(output)
    if info:
        info.setDuration(test_duration, test_mode)

class IterationInfo:
    # global info
    total_ts_retest_all = 0
    total_count_retest_all = 0
    total_failed_all = 0
    total_ts_retest_lazyut = 0
    total_count_retest_lazyut = 0
    total_failed_lazyut = 0
    @staticmethod
    def addFailed(mode):
        if mode == TestMode.retest_all:
            IterationInfo.total_failed_all += 1
        elif mode == TestMode.test_affected:
            IterationInfo.total_failed_lazyut += 1
        else:
            print('addFailed::unknown')
    # instance info
    def setCommit(self, commit):
        self.commit = commit
    def setDurationAll(self, ts):
        self.retest_all_ts = ts
        IterationInfo.total_ts_retest_all += ts
        IterationInfo.total_count_retest_all += 1
    def setDurationLazyut(self, ts):
        self.retest_lazyut_ts = ts
        IterationInfo.total_ts_retest_lazyut += ts
        IterationInfo.total_count_retest_lazyut += 1
    def setDurationLazyutAnalysis(self, ts):
        self.lazyut_analysis = ts
    def setDuration(self, ts, mode):
        if mode == TestMode.retest_all:
            self.setDurationAll(ts)
        elif mode == TestMode.test_affected:
            self.setDurationLazyut(ts)
        else:
            print('setDuration::unknown')
    

list_iteration_info = []
def comparing_iteration(pro_dir, commit = None):
    # clear local changes for project
    execute('git reset --hard', cwd = pro_dir, shell = True)
    # build default, then build LazyUT in the same build directory
    profile_info = IterationInfo()
    profile_info.setCommit(commit)
    build_project(pro_dir, TestMode.retest_all, info = profile_info)
    build_project(pro_dir, TestMode.test_affected, info = profile_info)
    list_iteration_info.append(profile_info)

def run_on_commit(pro_dir, commit = None):
    # run re-test all and re-test affected (LazyUT) several times
    # and store evaluation times
    for i in range(0, 3):
        print('iter #%d:' % i)
        comparing_iteration(pro_dir, commit = commit)
    # evaluate average testing times
    #  -- re-test all
    avg_retest_all = 0
    if list_iteration_info:
        avg_retest_all = IterationInfo.total_ts_retest_all / IterationInfo.total_count_retest_all
    #  -- re-test affected (LazyUT)
    avg_retest_lazyut = 0
    if list_iteration_info:
        avg_retest_lazyut = IterationInfo.total_ts_retest_lazyut / IterationInfo.total_count_retest_lazyut
    
    output = 're-test all {:.5}[sec]'.format(avg_retest_all)
    if avg_retest_lazyut == 0:
        output += ' re-test affected --SKIPPED'
    else:
        output += ' re-test affected (LazyUT) {:.5}[sec]'.format(avg_retest_lazyut)
    print(output)
#     append_out_file(str(avg_retest_all) + ' ' + str(avg_retest_lazyut))
    
def git_checkout(pro_dir, commit):
    execute('git reset --hard', cwd = pro_dir)
    execute('git checkout {}'.format(commit), cwd = pro_dir)
# -- SCRIPT --
# clears output.txt
clear_out_file()
# clears lazyut/ directory
removeFilesLazyUT(project_dir)
# get the list of commits
rev_list_str = check_output('git rev-list  -n 1000 --after="{}" {}'.format(
                    first_commit_date, branch), cwd = project_dir, shell=True)
rev_list = rev_list_str.splitlines()[::-1]
print('total commits: %d' %len(rev_list))
# iterate over commits, starting with 'the first commit'
for commit_bytes in rev_list:
    commit = commit_bytes.decode('utf-8')
    print('evaluate commit {}'.format(commit))
    # checkout to commit
    git_checkout(project_dir, commit)
    # evaluate comparing for commit
    run_on_commit(project_dir, commit = commit)
    # move LazyUT files from lazyut/files/ to lazyut/last_version/files/
    moveFilesLazyUT(project_dir)
print('script finished %s' % str(datetime.datetime.now()) )

total commits: 20
evaluate commit 3805af18a293e0b058fac305c68dec5d0ea4e733
iter #0:


In [6]:
for x in list_iteration_info:
    x.print()

18.103623656946183
20.230809355026395
19.84677700280008
19.707556721141373
19.9834725213168
20.981669507174956
0.3570109871270688
0.7609694679940731
1.2567470401045284
1.167634432165869
0.3604072221987735
20.2374234254803
0.3378114372608252
20.52016339117654
20.04059785328718
20.837955424829488
20.97445231898564
20.764852906284432
20.560651970879917
0.3476771514979191
20.96295625494713
20.362783771724935
19.647358145077305
19.102693624037784
19.73335494656203
0.6654643553792994
0.8259845338634477
0.600328545218872
0.6369807769042382
0.5090908377424057
0.5966026782298286
0.35523000881948974
0.7811204828813061
0.5747841846714437
0.47222368121401814
0.6392712961242069
1.5645025063495268
19.586483099119505
1.4060017761494237
0.6166247985838709
19.5517440607382
0.5617048611056816
0.3463130428590375
0.38158184682106366
0.3365433203343855
19.497089760701783
19.25277787336563
0.8434239338112093
0.3344046370639262
